In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from problems import *
from solvers import *

In [ ]:
# problem = QUBO(d=50, n=2, seed=0, mode="normal", loc=0, scale=1)
# solver = RandomSearch(problem=problem, budget=10, seed=0)
# solver = PSO(problem=problem, budget=10, seed=0)
# solver = PROTES(problem=problem, budget=10, seed=0)
# solver.optimize()

In [ ]:
from scripts.create_problem import load_problem_set
from scripts.run_solver import load_results

In [ ]:
problems = load_problem_set('data/normal/QUBO__mode_normal__loc_-5__scale_1/train')

In [ ]:
for problem in problems:
    print(problem.info)

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
import os

In [ ]:
def average_results(results):
    av_results = {'m': [], 'y (mean)': [], 'y (std)': []}
    for problem, logs in results.items():
        m = logs['m_list'][0]
        y = np.minimum.accumulate(logs['y_list'], axis=-1)
        y_mean = np.mean(y, axis=0)
        y_std = np.std(y, axis=0)
        av_results['m'].append(m)
        av_results['y (mean)'].append(y_mean)
        av_results['y (std)'].append(y_std)

    av_results['m'] = av_results['m'][0]
    av_results['y (mean)'] = np.mean(av_results['y (mean)'], axis=0)
    av_results['y (std)'] = np.mean(av_results['y (std)'], axis=0)
    return av_results

In [ ]:
read_dir = 'results/normal'
problem_list = os.listdir(read_dir)
solver_list = ['RandomSearch', 'PSO', 'PROTES']
suffix = 'test'

# plt.figure()
for solver in solver_list:
    av_results = defaultdict(list)
    for problem in problem_list:
        read_path = os.path.join(read_dir, problem, suffix, solver)
        results = load_results(read_path)
        for k, v in average_results(results).items():
            av_results[k].append(v)

    av_results['m'] = av_results['m'][0]
    av_results['y (mean)'] = np.mean(av_results['y (mean)'], axis=0)
    av_results['y (std)'] = np.mean(av_results['y (std)'], axis=0)

    plt.plot(av_results['m'], av_results['y (mean)'], label=solver)
    plt.fill_between(
        av_results['m'], 
        av_results['y (mean)'] - av_results['y (std)'], 
        av_results['y (mean)'] + av_results['y (std)'], 
        alpha=0.3
    )
plt.legend()
plt.show()

In [ ]:
from train_dpt_ import ProblemDataModule, DPTSolver, L
from utils import load_config

In [ ]:
config = load_config('config.yaml')
datamodule = ProblemDataModule(config)
datamodule.setup()
datamodule.test_dataloader().dataset[0]

In [ ]:
batch = next(iter(datamodule.train_dataloader()))

In [ ]:
batch.keys()

In [ ]:
batch["actions"].max()

In [ ]:
model.model.action_dim

In [ ]:
model = DPTSolver(config)
model.configure_model()

In [ ]:
rewards = model.reward_model.offline(
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"]
)
outputs = model.model(
    query_state=batch["query_state"],
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"],
    rewards=rewards,
)

In [ ]:
trainer = L.Trainer(
    logger=None,
    precision=config["precision"] if config["strategy"] != "fsdp" else None,
    max_epochs=config["max_epochs"],
    log_every_n_steps=config["log_every_n_steps"],
    default_root_dir=config["wandb_params"]["save_dir"],
    enable_model_summary=True,
    use_distributed_sampler=False,
    # strategy=config["strategy"]
)